In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction")
import pandas as pd
from pca_fts.PcaWeightedMVFTS import PcaWeightedMVFTS
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Aux functions

In [2]:
def sample_first_prows(data, perc=0.75):
    return data.head(int(len(data)*(perc)))

In [3]:
# convert series to supervised learning
def series_to_supervised_miso(data, n_in, n_out, endog_var='Global_active_power', dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df[endog_var].shift(-i))
		if i == 0:
			names += [(endog_var+'(t)')]
		else:
			names += [(endog_var+'(t+%d)' % (i))]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

## Dataset Household

In [21]:
filepath = '/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/household_power_consumption.csv'
data = pd.read_csv(filepath, sep = ";")
data = data.drop(labels=['Time','Date'], axis=1)
data = data.loc[0: : 30] # 30 minutes
data.dropna(inplace = True)
data = clean_dataset(data)
data.head()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Global_active_power  Global_reactive_power  Voltage  Global_intensity  \
0                  4.216                  0.418   234.84              18.4   
30                 2.720                  0.000   235.06              11.6   
60                 3.452                  0.000   235.20              15.2   
90                 4.298                  0.000   232.39              18.4   
120                3.262                  0.052   232.64              14.0   

     Sub_metering_1  Sub_metering_2  Sub_metering_3  
0               0.0             1.0            17.0  
30              0.0             0.0            17.0  
60              0.0             1.0            17.0  
90              0.0             1.0            16.0  
120             0.0             0.0            17.0

In [22]:
data['Global_active_power'] = pd.to_numeric(data['Global_active_power'],errors='coerce')
data['Global_reactive_power'] = pd.to_numeric(data['Global_reactive_power'],errors='coerce')
data['Voltage'] = pd.to_numeric(data['Voltage'],errors='coerce')
data['Global_intensity'] = pd.to_numeric(data['Global_intensity'],errors='coerce')
data['Sub_metering_1'] = pd.to_numeric(data['Sub_metering_1'],errors='coerce')
data['Sub_metering_2'] = pd.to_numeric(data['Sub_metering_2'],errors='coerce')
data['Sub_metering_3'] = pd.to_numeric(data['Sub_metering_3'],errors='coerce')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68308 entries, 0 to 2075250
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Global_active_power    68308 non-null  float64
 1   Global_reactive_power  68308 non-null  float64
 2   Voltage                68308 non-null  float64
 3   Global_intensity       68308 non-null  float64
 4   Sub_metering_1         68308 non-null  float64
 5   Sub_metering_2         68308 non-null  float64
 6   Sub_metering_3         68308 non-null  float64
dtypes: float64(7)
memory usage: 4.2 MB


In [24]:
round(data.describe(),3)

Global_active_power  Global_reactive_power    Voltage  \
count            68308.000              68308.000  68308.000   
mean                 1.092                  0.124    240.853   
std                  1.059                  0.113      3.219   
min                  0.078                  0.000    225.320   
25%                  0.308                  0.048    239.050   
50%                  0.600                  0.100    241.030   
75%                  1.530                  0.194    242.870   
max                 10.290                  1.094    253.360   

       Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
count         68308.000       68308.000       68308.000       68308.000  
mean              4.629           1.132           1.293           6.459  
std               4.451           6.211           5.785           8.428  
min               0.200           0.000           0.000           0.000  
25%               1.400           0.000           0.000           0.000  
50%               2.600           0.000           0.000           1.000  
75%               6.400           0.000           1.000          17.000  
max              44.600          87.000          79.000          31.000

### Dataset Household: Preprocessing, Lag = 1, t+1

In [25]:
# frame as supervised learning
data = series_to_supervised_miso(data, 1, 1,endog_var='Global_active_power')
data.head()

Global_active_power(t-1)  Global_reactive_power(t-1)  Voltage(t-1)  \
30                      4.216                       0.418        234.84   
60                      2.720                       0.000        235.06   
90                      3.452                       0.000        235.20   
120                     4.298                       0.000        232.39   
150                     3.262                       0.052        232.64   

     Global_intensity(t-1)  Sub_metering_1(t-1)  Sub_metering_2(t-1)  \
30                    18.4                  0.0                  1.0   
60                    11.6                  0.0                  0.0   
90                    15.2                  0.0                  1.0   
120                   18.4                  0.0                  1.0   
150                   14.0                  0.0                  0.0   

     Sub_metering_3(t-1)  Global_active_power(t)  
30                  17.0                   2.720  
60                  17.0                   3.452  
90                  17.0                   4.298  
120                 16.0                   3.262  
150                 17.0                   3.214

In [26]:
data

Global_active_power(t-1)  Global_reactive_power(t-1)  Voltage(t-1)  \
30                          4.216                       0.418        234.84   
60                          2.720                       0.000        235.06   
90                          3.452                       0.000        235.20   
120                         4.298                       0.000        232.39   
150                         3.262                       0.052        232.64   
...                           ...                         ...           ...   
2075130                     1.716                       0.128        238.42   
2075160                     1.556                       0.064        237.56   
2075190                     1.120                       0.082        238.91   
2075220                     1.468                       0.134        238.44   
2075250                     1.082                       0.136        240.32   

         Global_intensity(t-1)  Sub_metering_1(t-1)  Sub_metering_2(t-1)  \
30                        18.4                  0.0                  1.0   
60                        11.6                  0.0                  0.0   
90                        15.2                  0.0                  1.0   
120                       18.4                  0.0                  1.0   
150                       14.0                  0.0                  0.0   
...                        ...                  ...                  ...   
2075130                    7.2                  0.0                  0.0   
2075160                    6.6                  0.0                  0.0   
2075190                    4.6                  0.0                  0.0   
2075220                    6.2                  0.0                  0.0   
2075250                    4.6                  0.0                  1.0   

         Sub_metering_3(t-1)  Global_active_power(t)  
30                      17.0                   2.720  
60                      17.0                   3.452  
90                      17.0                   4.298  
120                     16.0                   3.262  
150                     17.0                   3.214  
...                      ...                     ...  
2075130                  0.0                   1.556  
2075160                  0.0                   1.120  
2075190                  0.0                   1.468  
2075220                  0.0                   1.082  
2075250                  0.0                   0.946  

[68307 rows x 8 columns]

## Model: PcaWeightedMVFTS - PCA-MVFTS

In [27]:
pca_wmvfts = PcaWeightedMVFTS(n_components = 2,
                       endogen_variable = 'Global_active_power(t)',
                       n_part = 50)

In [32]:
def sliding_window_pca(data,n_windows,train_size,dim,fs):

    result = {
         "window": [],
         "rmse": [],
         "mae": [],
         "mape": [],
         "smape": []
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
        if len(ttest) > 0:

            pca_wmvfts = PcaWeightedMVFTS(n_components = dim,
                       endogen_variable = 'Global_active_power(t-1)',
                       n_part = fs)
            
            data_train = ttrain.loc[:,'Global_active_power(t-1)':'Sub_metering_3(t-1)']
            data_test = ttest.loc[:,'Global_reactive_power(t-1)':'Global_active_power(t)']
        
            #ALtera o nome da columa porque o modelo utiliza esse label como variável endogena 
            data_test.rename(columns = {'Global_active_power(t)': 'Global_reactive_power(t-1)'}, inplace = True)
            
            model, pca_reduced_train = pca_wmvfts.run_train_model(ttrain)
            forecast, forecast_self, pca_reduced_test = pca_wmvfts.run_test_model(model,ttest)
            
            #Teste e cálculo dos erros da previsão  
            #valor use Global_reactive_power(t-1) é o Global_reactive_power(t) --> label igual por causa do modelo 
            rmse = Measures.rmse(pca_reduced_test['Global_active_power(t-1)'],forecast)
            mape = Measures.mape(pca_reduced_test['Global_active_power(t-1)'],forecast)
            smape = Measures.smape(pca_reduced_test['Global_active_power(t-1)'],forecast)
            
            forecast = pd.DataFrame(forecast)
            forecast.fillna(forecast.mean(),inplace=True)
            forecast = np.array(forecast).reshape(-1)
            mae = mean_absolute_error(pca_reduced_test['Global_active_power(t-1)'], forecast)
            
            result["rmse"].append(round(rmse,3))
            result["mape"].append(round(mape,3))
            result["smape"].append(round(smape,3))
            result["mae"].append(round(mae,3))
            result["window"].append(ct)
        
    measures = pd.DataFrame(result)
    return measures



In [35]:
dimensions = [2,3,4]
fuzzy_sets = [10,20,30,40,50]

final_result = {
    "dimensions": [],
    "fuzzy_sets": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "smape": []
}

n_windows = 30
train_size = 0.75

for dim in dimensions:
    for fs in fuzzy_sets:
        measures = sliding_window_pca(data,n_windows,train_size,dim,fs)
        final_result["dimensions"].append(dim)
        final_result["fuzzy_sets"].append(fs)
        
        rmse = round(statistics.mean(measures['rmse']),3)
        mape = round(statistics.mean(measures['mape']),3)
        smape = round(statistics.mean(measures['smape']),3)
        mae = round(statistics.mean(measures['mae']),3)
        
        final_result["rmse"].append(rmse)
        final_result["mape"].append(mape)
        final_result["smape"].append(smape)
        final_result["mae"].append(mae)
        
        print(f'Results: {(dim,fs,rmse,mae,mape,smape)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics PCA-WMVFTS (test): ")
final_measures_pca

Results: (2, 10, 0.515, 0.391, 94.095, 25.257)
Results: (2, 20, 0.363, 0.259, 52.431, 16.651)
Results: (2, 30, 0.29, 0.192, 37.07, 12.827)
Results: (2, 40, 0.26, 0.153, 30.576, 10.683)
Results: (2, 50, 0.233, 0.132, 26.765, 9.44)
Results: (3, 10, 0.451, 0.344, 85.703, 23.333)
Results: (3, 20, 0.258, 0.177, 38.198, 12.65)
Results: (3, 30, 0.193, 0.11, 24.039, 8.403)
Results: (3, 40, 0.154, 0.077, 17.253, 6.053)
Results: (3, 50, 0.118, 0.052, 11.758, 4.172)
Results: (4, 10, 0.385, 0.291, 75.353, 20.861)
Results: (4, 20, 0.173, 0.105, 23.704, 8.143)
Results: (4, 30, 0.111, 0.054, 12.742, 4.602)
Results: (4, 40, 0.073, 0.029, 6.835, 2.531)
Results: (4, 50, 0.047, 0.017, 3.746, 1.418)
Statistics PCA-WMVFTS (test): 


dimensions  fuzzy_sets   rmse    mape    mae   smape
0            2          10  0.515  94.095  0.391  25.257
1            2          20  0.363  52.431  0.259  16.651
2            2          30  0.290  37.070  0.192  12.827
3            2          40  0.260  30.576  0.153  10.683
4            2          50  0.233  26.765  0.132   9.440
5            3          10  0.451  85.703  0.344  23.333
6            3          20  0.258  38.198  0.177  12.650
7            3          30  0.193  24.039  0.110   8.403
8            3          40  0.154  17.253  0.077   6.053
9            3          50  0.118  11.758  0.052   4.172
10           4          10  0.385  75.353  0.291  20.861
11           4          20  0.173  23.704  0.105   8.143
12           4          30  0.111  12.742  0.054   4.602
13           4          40  0.073   6.835  0.029   2.531
14           4          50  0.047   3.746  0.017   1.418

In [38]:
final_measures_pca.to_csv (r'pca_wmvfts_dim_fsets_hpc.csv', index = False, header=True)